In [1]:
# Merge all recorded data

from hoboreader import HoboReader
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from pathlib import Path
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import helper_functions
%matplotlib inline


pd.set_option('mode.chained_assignment', None)

# Set up figure size and DPI for screen demo
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 150


# DL1	Stone Garden - Tjaden Hall
# DL2	PSB Parking Lot
# DL3	Olin Library
# DL4	Milstein Hall Roof
# DL5	Game Farm Road


S1 = HoboReader("DL1_Stone_Garden_Tjaden_Hall.csv")
S2 = HoboReader("DL2_PSB_Parking_Lot.csv")
S3 = HoboReader("DL3_Olin_Library.csv")
S4 = HoboReader("DL4_Milstein_Hall_Roof.csv")
S5 = HoboReader("DL5_Game_Farm_Road_20192020v5.csv")


df1 = S1.get_dataframe()
df2 = S2.get_dataframe()
df3 = S3.get_dataframe()
df4 = S4.get_dataframe()
df5 = S5.get_dataframe()

In [2]:
# DF5 (GAME FARM ROAD) HAS TO BE SHIFTED 1HR BACK BECAUSE IT NATIVELY RECORDS DST, OUR STATIONS DO NOT.
# THE OTHER STATIONS ARE NOT SHIFTED (HENCE 0 VALUE FOR PERIODS), JUST THERE FOR SANITY

df5 = df5.shift(periods=(-1), fill_value=0)
df4 = df4.shift(periods=(0), fill_value=0)
df3 = df3.shift(periods=(0), fill_value=0)
df2 = df2.shift(periods=(0), fill_value=0)
df1 = df1.shift(periods=(0), fill_value=0)

# Resample subhourly data to 1 hourly graph (OUR STATIONS RECORD IN 10MIN INTERVALS, GFR IN 1HR, RESAMPLE FOR EASY COMPARISON. CAN TURN OFF)
df1 = df1.resample('H').mean()
df2 = df2.resample('H').mean()
df3 = df3.resample('H').mean()
df4 = df4.resample('H').mean()
df5 = df5.resample('H').mean()

# CONVERT IMPERIAL TO METRIC UNITS

def ftoc(f):
    return (f - 32) * 5.0/9.0

def mphtoms(mph):
    return mph * 0.44704

def f(variable_id, d, m, y):

    start_time = str(y)+'-'+str(m)+'-'+str(d)+' 07:00:00-00:00'
    end_time = str(y)+'-'+str(m)+'-'+str(d)+' 23:00:00-00:00'

    df1_ = df1.loc[start_time:end_time]
    df2_ = df2.loc[start_time:end_time]
    df3_ = df3.loc[start_time:end_time]
    df4_ = df4.loc[start_time:end_time]
    df5_ = df5.loc[start_time:end_time]

    # Translate to °C

  
    df1_["Temp"] = df1_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df2_["Temp"] = df2_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df3_["Temp"] = df3_["Temp"].apply(lambda x: ftoc(x), axis=1)
    df4_["Temp"] = df4_["Temp"].apply(lambda x: ftoc(x), axis=1)

    df1_["MRT"] = df1_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df2_["MRT"] = df2_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df3_["MRT"] = df3_["MRT"].apply(lambda x: ftoc(x), axis=1)
    df4_["MRT"] = df4_["MRT"].apply(lambda x: ftoc(x), axis=1)
    

    # Translate to m/s

    df1_["Wind Speed"] = df1_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df2_["Wind Speed"] = df2_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df3_["Wind Speed"] = df3_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    df4_["Wind Speed"] = df4_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df5_["Wind Speed"] = df5_["Wind Speed"].apply(lambda x: mphtoms(x), axis=1)
    
    df1_["Gust Speed"] = df1_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df2_["Gust Speed"] = df2_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df3_["Gust Speed"] = df3_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    df4_["Gust Speed"] = df4_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
    #df5_["Gust Speed"] = df5_["Gust Speed"].apply(lambda x: mphtoms(x), axis=1)
   

    # df1.index

    # Print Headers
    # print(df1.columns.values)

    # Resize figure
    custom_figsize = (20, 10)

    # Change lineweights
    lineweights = 1.5

    variables = ["Wind Speed", "MRT", "Temp", "RH", "Solar Radiation",
        "Wind Speed", "Gust Speed", "Wind Direction", "DewPt"]

    variable = variables[variable_id]

    # Plot both temp and mrt when temp is selected

    if variable == "MRT" or variable == "DewPt" or variable == "Gust Speed":
        ax = df1_.plot(y=variable, lw=1, figsize=custom_figsize, color='red')
        df2_.plot(y=variable, lw=1, figsize=custom_figsize,
                   color='blue', ax=ax)
        df3_.plot(y=variable, lw=1, figsize=custom_figsize,
                   color='green', ax=ax)
        df4_.plot(y=variable, lw=1, figsize=custom_figsize, color='aqua', ax=ax)

    elif variable == "Temp":
        # Plot two dataframes into one plot
        ax = df1_.plot(y=variable, lw=2, figsize=custom_figsize, color='red')
        df2_.plot(y=variable,    lw=2, figsize=custom_figsize, color='blue', ax=ax)
        df3_.plot(y = variable,     lw=2, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable,     lw=2, figsize=custom_figsize,color='aqua',ax=ax)
        df5_.plot(y = variable,     lw=3.5, figsize=custom_figsize,color='black',ax=ax)

        ax = df1_.plot(y = "MRT", lw=1, figsize=custom_figsize,style = ['--'],color='red', ax=ax)
        df2_.plot( y = "MRT",     lw=1, figsize=custom_figsize, style = ['--'],color='blue',ax=ax)
        df3_.plot(y = "MRT",      lw=1, figsize=custom_figsize, style = ['--'],color='green',ax=ax)
        df4_.plot(y = "MRT",      lw=1, figsize=custom_figsize, style = ['--'],color='aqua',ax=ax)      
                   
    else:  
         # Plot two dataframes into one plot
        ax = df1_.plot(y = variable, lw=1, figsize=custom_figsize,color='red')
        df2_.plot( y = variable, lw=1, figsize=custom_figsize,color='blue',ax=ax)
        df3_.plot(y = variable, lw=1, figsize=custom_figsize,color='green',ax=ax)
        df4_.plot(y = variable, lw=1, figsize=custom_figsize,color='aqua',ax=ax)
        df5_.plot(y = variable, lw=2, figsize=custom_figsize,color='black',ax=ax)
     
     # Legend
    ax.legend([
       "DL1 - Stone Garden - Tjaden Hall", 
       "DL2 - PSB Parking Lot",
       "DL3 - Olin Library",
       "DL4 - Milstein Hall Roof ",
       "DL5 - Game Farm Road"
       
       ],loc='upper left', frameon=False)

    

    
    
    # Fix xticks
    
    years = mdates.YearLocator()   # every year
    months = mdates.MonthLocator()  # every month
    days = mdates.DayLocator()  # every minute
    hours = mdates.HourLocator()  # every minute
    minutes = mdates.MinuteLocator()  # every minute
    
    days_fmt = mdates.DateFormatter('%d')
    hours_fmt = mdates.DateFormatter('%H:00')
    
    # format the ticks
    ax.xaxis.set_major_locator(hours)
    ax.xaxis.set_major_formatter(hours_fmt)
    ax.xaxis.set_minor_locator(hours)
    
    # Labels
    plt.ylabel(variable, fontname="Consolas", fontsize=20)
    plt.xlabel("UTC Time", fontname="Consolas", fontsize=20)
         
    # Title
    ax.set_title(str(start_time.split(' ')[0]+" "+variable), fontname="Consolas", fontsize=28)
    
    # Save PDF
    # plot_graph(plt, start_time, variable)

    
# Variable to plot
# 1 = "MRT °C"
# 2 = "Temp °C"
# 3 = "RH"
# 4 = "Solar Radiation"
# 5 = "Wind Speed"
# 6 = "Gust Speed"
# 7 = "Wind Direction"
# 8 = "DewPt"
    
    
interact(f,variable_id=(1,8,1), d=(1,31,1), m=(1,12,1), y = (2019,2021,1))

#04142020,f/c issue, 05052020 no f/c issue, 

interactive(children=(IntSlider(value=4, description='variable_id', max=8, min=1), IntSlider(value=16, descrip…

<function __main__.f(variable_id, d, m, y)>

In [3]:
# GAME FARM ROAD_ITHACA STATION
import time
import math
import pandas as pd
#import pvlib
from hoboreader import HoboReader
S5 = HoboReader("DL5_Game_Farm_Road_2019.csv")
df5 = S5.get_dataframe()


#!pip install pvlib
# https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.reindl.html?highlight=reindl#pvlib.irradiance.reindl

# conda install -c pvlib pvlib

# Create date series using date_range() function
date_series = pd.date_range('01/01/2019', periods=8760, freq='H')
# print(date_series)

# altitude GFR Station
#altitude = 294

# lat, lon taken from GFR
lon = 42.449  # north-positive
lat = -76.449  # west-negative #west of england

#pos = pvlib.solarposition.get_solarposition(date_series,lat, lon)
# pos


solar_constant_e = 1367.0
DTR = 0.017453292  # Pi / 180
RTD = 57.2957795  # 180 / Pi


time_zone = 5*15  # west-positive


def month_and_day_to_julian_day(month, day):
    if (month == 1):
        return day
    if (month == 2):
        return day + 31
    if (month == 3):
        return day + 59
    if (month == 4):
        return day + 90
    if (month == 5):
        return day + 120
    if (month == 6):
        return day + 151
    if (month == 7):
        return day + 181
    if (month == 8):
        return day + 212
    if (month == 9):
        return day + 243
    if (month == 10):
        return day + 273
    if (month == 11):
        return day + 304
    if (month == 12):
        return day + 334
    else:
        print("bad month")


def solar_elev_azi_ecc(latitude,
                       longitude,
                       time_zone,
                       jday,
                       time,
                       solar_time=0):
    # /*  angles in degrees, times in hours  */

    # /*  solar elevation and azimuth formulae from sun.c  */
    if (solar_time == 1):
        sol_time = time
    if (solar_time == 0):
        sol_time = time + 0.170 * math.sin(
            (4 * math.pi / 373) * (jday - 80)) - 0.129 * math.sin(
                (2 * math.pi / 355) *
                (jday - 8)) + 12 / 180.0 * (time_zone - longitude)

    solar_declination = RTD * 0.4093 * math.sin(
        (2 * math.pi / 368) * (jday - 81))
    jday_angle = 2 * math.pi * (jday - 1) / 365

    solar_elevation = RTD * math.asin(
        math.sin(latitude * DTR) * math.sin(solar_declination * DTR) -
        math.cos(latitude * DTR) * math.cos(solar_declination * DTR) *
        math.cos(sol_time * (math.pi / 12)))

    solar_azimuth = RTD * (-math.atan2(
        math.cos(solar_declination * DTR) * math.sin(sol_time *
                                                     (math.pi / 12)),
        -math.cos(latitude * DTR) * math.sin(solar_declination * DTR) -
        math.sin(latitude * DTR) * math.cos(solar_declination * DTR) *
        math.cos(sol_time * (math.pi / 12))))

    # /*  eccentricity_correction formula used in genjdaylit.c */

    eccentricity_correction = 1.00011 + 0.034221 * math.cos(
        jday_angle) + 0.00128 * math.sin(jday_angle) + 0.000719 * math.cos(
            2 * jday_angle) + 0.000077 * math.sin(2 * jday_angle)

    return solar_elevation, solar_azimuth, eccentricity_correction


def diffuse_fraction(irrad_glo, solar_elevation, eccentricity_correction):
    dif_frac = 0

    if (solar_elevation > 0):
        irrad_ex = solar_constant_e * eccentricity_correction * math.sin(
            DTR * solar_elevation)
    else:
        irrad_ex = 0

    if (irrad_ex > 0):
        index_glo_ex = irrad_glo / irrad_ex
    else:
        return 0

    if (index_glo_ex < 0):
        print("negative irrad_glo in diffuse_fraction_th\n")
    if (index_glo_ex > 1):
        index_glo_ex = 1

    if (index_glo_ex <= 0.3):
        dif_frac = 1.02 - 0.254 * index_glo_ex + 0.0123 * math.sin(
            DTR * solar_elevation)
    if (dif_frac > 1):
        dif_frac = 1

    if (0.3 < index_glo_ex < 0.78):
        dif_frac = 1.4 - 1.749 * index_glo_ex + 0.177 * math.sin(
            DTR * solar_elevation)
    if (dif_frac > 0.97):
        dif_frac = 0.97
    if (dif_frac < 0.1):
        dif_frac = 0.1

    if (index_glo_ex >= 0.78):
        dif_frac = 0.486 * index_glo_ex - 0.182 * math.sin(
            DTR * solar_elevation)
    if dif_frac < 0.1:
        dif_frac = 0.1

    return dif_frac


def calc_split(month, day, time, irrad_glo):
    jday = month_and_day_to_julian_day(month, day)

    if (irrad_glo < 0 or irrad_glo > solar_constant_e
            ):  # / * check irradiances and exit if necessary * /
        irrad_glo = solar_constant_e

    # no object orientation yet

    solar_elevation, solar_azimuth, eccentricity_correction = solar_elev_azi_ecc(
        lat, lon, time_zone, jday, time)

    #

    irrad_dif = diffuse_fraction(irrad_glo, solar_elevation,
                                 eccentricity_correction) * irrad_glo

    if (solar_elevation > 5.0):

        irrad_beam_nor = (irrad_glo - irrad_dif) * 1.0 / math.sin(
            DTR * solar_elevation)

    else:
        irrad_beam_nor = 0
        irrad_dif = irrad_glo

    if (irrad_beam_nor > solar_constant_e):
        irrad_beam_nor = solar_constant_e
        irrad_dif = irrad_glo - irrad_beam_nor * math.sin(
            DTR * solar_elevation)

    #print(month, day, round(time, 2), round(irrad_beam_nor, 2), round(irrad_dif, 2))

    return irrad_beam_nor, irrad_dif


#print("month, day, time, GHR\n")

%timeit calc_split(4, 22, 8.333333333, 107)



9.89 µs ± 740 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
import time
from tqdm import tqdm



def calc_split_df(df):
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['DHR'] = 0
    df['DNI'] = 0

    i = 0
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        GHR = df.iloc[i, df.columns.get_loc('Solar Radiation')].values[0]
        DNI, DHR = calc_split(df.month[i], df.day[i], df.hour[i], GHR)
        #print(i,  GHR, DNI, DHR)
        df.iloc[i, df.columns.get_loc('DHR')] = DHR
        df.iloc[i, df.columns.get_loc('DNI')] = DNI
        i += 1

    return df


df5 = calc_split_df(df5)

100%|██████████████████████████████████████████████████████████████████████████████| 8760/8760 [02:10<00:00, 67.32it/s]


# Variables in the epw that we need


# From GFR
average temperature (F),
 relative humidity (%), 
solar radiation (langleys), 
10 meter wind speed (mph), 
wind direction (degrees), 
precipitation (inches)

# To EPW

average temperature (°C),
relative humidity (%), 
GHI  (Wh/m2), 
DNI  (Wh/m2), 
DHI  (Wh/m2), 
10 meter wind speed (m/s), 
wind direction (degrees), 
precipitation (cm or mm)


# Make the 2019 file
# Make the 2020 file and replace half of the year with the values from 2019

 df5_2020[start_time:end_time] = df5_2019.loc[start_time:end_time]



In [5]:
# GAME FARM ROAD_ITHACA STATION

import pandas as pd
from pandas import read_csv
import csv
import unittest
import numpy as np
from pprint import pprint
from setuptools import setup, find_packages
from codecs import open
from os import path
from epw import epw

#S5 = HoboReader("DL5_Game_Farm_Road_2019.csv")
#df5 = S5.get_dataframe()

#There are about 35 variables in the core weather data. However, not all of them are used by EnergyPlus. Actually, despite of date and time columns, only 13 columns are used:
#dry bulb temperature 
#dew point temperature 
#relative humidity 
#atmospheric pressure 
#horizontal infrared radiation intensity from sky 
#direct normal radiation 
#diffuse horizontal radiation 
#wind direction 
#wind speed 
#present weather observation 
#present weather codes 
#snow depth 
#liquid precipitation depth

# https://www.radiance-online.org/pipermail/radiance-general/2015-April/010841.html

# Daniel,

# You should be able to generate the .wea file from diffuse horizontal
# radiation and direct normal radiation values. The structure of the lines
# after the heading in .wea file is:

# month day hour directNormalRadiation diffuseHorizontalRadiation

# Template

template = epw()
template.read('USA_NY_Ithaca_Template.epw')
template.dataframe.columns


# Real file

ithaca_2019 = epw()
# ithaca_2019.headers['LOCATION'][0]='Ithaca_NY'

ithaca_2019.headers = {'LOCATION': ['Ithaca',
                                    'NY',
                                    'USA',
                                    'TMY3',
                                    '725190',
                                    '42.438934',
                                    '-76.492435',
                                    '-5.0',
                                    '125.0'],
                       'DESIGN CONDITIONS': ['0'],
                       'TYPICAL/EXTREME PERIODS': ['0'],
                       'GROUND TEMPERATURES': ['3',
                                               '0.5',
                                               '',
                                               '',
                                               '',
                                               '2.44',
                                               '4.62',
                                               '9.34',
                                               '15.56',
                                               '21.51',
                                               '25.58',
                                               '26.67',
                                               '24.44',
                                               '19.55',
                                               '13.31',
                                               '7.40',
                                               '3.42',
                                               '2',
                                               '',
                                               '',
                                               '',
                                               '5.55',
                                               '6.08',
                                               '8.78',
                                               '13.07',
                                               '17.76',
                                               '21.58',
                                               '23.50',
                                               '23.98',
                                               '20.18',
                                               '15.85',
                                               '11.17',
                                               '7.40',
                                               '4',
                                               '',
                                               '',
                                               '',
                                               '8.79',
                                               '8.41',
                                               '9.60',
                                               '12.12',
                                               '15.30',
                                               '18.27',
                                               '20.23',
                                               '20.66',
                                               '19.42',
                                               '16.85',
                                               '13.67',
                                               '10.73'],
                       'HOLIDAYS/DAYLIGHT SAVINGS': ['No', '0', '0', '0'],
                       'COMMENTS 1': [''],
                       'COMMENTS 2': [''],
                       'DATA PERIODS': ['1', '1', 'Data', 'Sunday', '1/1', '12/31']}

#print(ithaca_2019.headers)


#variables = ["Wind Speed","MRT","Temp","RH","Solar Radiation","Wind Speed","Gust Speed","Wind Direction","DewPoint" ]

ithaca_2019.dataframe['Year'] = np.array(np.ones(8760)*2019).astype(int)
ithaca_2019.dataframe['Month'] = template.dataframe['Month'].astype(int)
ithaca_2019.dataframe['Day'] = template.dataframe['Day'].astype(int)
ithaca_2019.dataframe['Hour'] = template.dataframe['Hour'].astype(int)
ithaca_2019.dataframe['Minute'] = template.dataframe['Minute'].astype(int)
ithaca_2019.dataframe['Data Source and Uncertainty Flags'] = np.array(
    np.zeros(8760)).astype(int)


# Actual file starts here
# 6
# .apply(lambda x: ftoc(x), axis=1).values.flatten()
ithaca_2019.dataframe['Dry Bulb Temperature'] = df5['Temp'].values.flatten()

#     Add this later (ADDED TO DL5_2019 FILE 07.22.2020)
# np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Dew Point Temperature'] = df5['DewPoint'].values.flatten()


ithaca_2019.dataframe['Relative Humidity'] = df5['RH'].values.flatten()
#
ithaca_2019.dataframe['Atmospheric Station Pressure'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Extraterrestrial Horizontal Radiation'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Extraterrestrial Direct Normal Radiation'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Horizontal Infrared Radiation Intensity'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Global Horizontal Radiation'] = df5['Solar Radiation'].values.flatten()
ithaca_2019.dataframe['Direct Normal Radiation'] = df5['DNI'].values.flatten()
ithaca_2019.dataframe['Diffuse Horizontal Radiation'] = df5['DHR'].values.flatten()

# Do we need this?
# https://www.radiance-online.org//pipermail/radiance-general/2013-January/009140.html ?
ithaca_2019.dataframe['Global Horizontal Illuminance'] = df5['Solar Radiation'].values.flatten()*179
ithaca_2019.dataframe['Direct Normal Illuminance'] = df5['DNI'].values.flatten()*179
ithaca_2019.dataframe['Diffuse Horizontal Illuminance'] = df5['DHR'].values.flatten()*179

ithaca_2019.dataframe['Zenith Luminance'] = np.array(
    np.zeros(8760)).astype(int)

ithaca_2019.dataframe['Wind Direction'] = df5['Wind Direction'].values.flatten(
).astype(int)
ithaca_2019.dataframe['Wind Speed'] = df5['Wind Speed'].values.flatten()

#not sure which one to use (skycover)
ithaca_2019.dataframe['Total Sky Cover'] = df5['SkyCover'].replace(' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
#     This is from Ithaca airport data (NWS, NOAA) (ADDED TO DL5_2019 FILE 07.22.2020)
ithaca_2019.dataframe['Opaque Sky Cover (used if Horizontal IR Intensity missing)'] = df5['SkyCover'].replace(' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
#

ithaca_2019.dataframe['Visibility'] = np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Ceiling Height'] = np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Present Weather Observation'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Present Weather Codes'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Precipitable Water'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Aerosol Optical Depth'] = np.array(
    np.zeros(8760)).astype(int)
#
ithaca_2019.dataframe['Snow Depth'] = np.array(np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Days Since Last Snowfall'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Albedo'] = np.array(np.zeros(8760)).astype(int)
#

# We have precipitation but lets not use it for now
ithaca_2019.dataframe['Liquid Precipitation Depth'] = np.array(
    np.zeros(8760)).astype(int)
ithaca_2019.dataframe['Liquid Precipitation Quantity'] = np.array(
    np.zeros(8760)).astype(int)

pd.options.display.max_columns = None
display(ithaca_2019.dataframe.loc[8:20])


ithaca_2019.write('GameFarmRoad2019.epw')

,Year,Month,Day,Hour,Minute,Data Source and Uncertainty Flags,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,Extraterrestrial Horizontal Radiation,Extraterrestrial Direct Normal Radiation,Horizontal Infrared Radiation Intensity,Global Horizontal Radiation,Direct Normal Radiation,Diffuse Horizontal Radiation,Global Horizontal Illuminance,Direct Normal Illuminance,Diffuse Horizontal Illuminance,Zenith Luminance,Wind Direction,Wind Speed,Total Sky Cover,Opaque Sky Cover (used if Horizontal IR Intensity missing),Visibility,Ceiling Height,Present Weather Observation,Present Weather Codes,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
8,2019,1,1,9,0,0,6.611111,2.611111,80.0,0,0,0,0,0.000,0.000000,0.00000,0.000,0.000000,0.00000,0,293,18.4,10,10,0,0,0,0,0,0,0,0,0,0,0
9,2019,1,1,10,0,0,4.444444,0.244444,79.0,0,0,0,0,2.326,0.118447,2.25622,416.354,21.202034,403.86338,0,313,13.3,10,10,0,0,0,0,0,0,0,0,0,0,0
10,2019,1,1,11,0,0,3.333333,-0.266667,82.0,0,0,0,0,22.097,1.114320,21.43409,3955.363,199.463273,3836.70211,0,319,13.3,10,10,0,0,0,0,0,0,0,0,0,0,0
11,2019,1,1,12,0,0,2.944444,0.344444,87.0,0,0,0,0,30.238,1.548070,29.33086,5412.602,277.104533,5250.22394,0,309,15.2,10,10,0,0,0,0,0,0,0,0,0,0,0
12,2019,1,1,13,0,0,2.055556,-0.144444,89.0,0,0,0,0,38.379,2.045162,37.22763,6869.841,366.084057,6663.74577,0,314,12.3,10,10,0,0,0,0,0,0,0,0,0,0,0
13,2019,1,1,14,0,0,1.500000,-0.900000,88.0,0,0,0,0,17.445,0.992241,16.92165,3122.655,177.611115,3028.97535,0,315,11.4,10,10,0,0,0,0,0,0,0,0,0,0,0
14,2019,1,1,15,0,0,1.388889,-1.011111,88.0,0,0,0,0,12.793,0.796554,12.40921,2289.947,142.583209,2221.24859,0,308,10.9,10,10,0,0,0,0,0,0,0,0,0,0,0
15,2019,1,1,16,0,0,1.222222,-1.177778,88.0,0,0,0,0,12.793,0.894213,12.40921,2289.947,160.064211,2221.24859,0,308,8.8,10,10,0,0,0,0,0,0,0,0,0,0,0
16,2019,1,1,17,0,0,1.222222,-1.977778,84.0,0,0,0,0,15.119,1.215456,14.66543,2706.301,217.566601,2625.11197,0,308,12.0,10,10,0,0,0,0,0,0,0,0,0,0,0
17,2019,1,1,18,0,0,0.444444,-2.955556,83.0,0,0,0,0,3.489,0.329609,3.38433,624.531,58.999999,605.79507,0,321,11.5,10,10,0,0,0,0,0,0,0,0,0,0,0


In [6]:
a = epw()
a.read(r"USA_NY_Ithaca_Template.epw")
pd.options.display.max_columns = None
display(a.dataframe.loc[8:20])

,Year,Month,Day,Hour,Minute,Data Source and Uncertainty Flags,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,Extraterrestrial Horizontal Radiation,Extraterrestrial Direct Normal Radiation,Horizontal Infrared Radiation Intensity,Global Horizontal Radiation,Direct Normal Radiation,Diffuse Horizontal Radiation,Global Horizontal Illuminance,Direct Normal Illuminance,Diffuse Horizontal Illuminance,Zenith Luminance,Wind Direction,Wind Speed,Total Sky Cover,Opaque Sky Cover (used if Horizontal IR Intensity missing),Visibility,Ceiling Height,Present Weather Observation,Present Weather Codes,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
8,2080,1,1,9,0,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,5.2,2.5,83,99797,174,1413,340,32,7,31,3617,0,3617,1117,310,0.9,10,10,9999,99999,9,999999999,100,999,999,99,NaN,NaN,NaN
9,2080,1,1,10,0,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,5.5,2.5,81,99797,355,1413,342,74,10,71,8399,383,8131,2702,0,0.0,10,10,9999,99999,9,999999999,100,999,999,99,NaN,NaN,NaN
10,2080,1,1,11,0,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,5.9,2.3,78,99697,485,1413,344,135,20,128,15138,1236,14414,4771,0,0.0,10,10,9999,99999,9,999999999,100,999,999,99,NaN,NaN,NaN
11,2080,1,1,12,0,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,6.3,2.2,75,99697,556,1413,346,125,12,120,14373,755,13871,4898,0,0.0,10,10,9999,99999,9,999999999,100,999,999,99,NaN,NaN,NaN
12,2080,1,1,13,0,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,6.8,2.0,72,99597,564,1413,348,139,15,133,15847,970,15222,5306,0,0.0,10,10,9999,99999,9,999999999,100,999,999,99,NaN,NaN,NaN
13,2080,1,1,14,0,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,6.6,2.1,73,99597,508,1413,347,133,18,127,15039,1109,14363,4856,10,0.9,10,10,9999,99999,9,999999999,100,999,999,99,NaN,NaN,NaN
14,2080,1,1,15,0,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,6.5,2.2,74,99597,391,1413,347,94,15,89,10565,691,10131,3338,30,1.7,10,10,9999,99999,9,999999999,100,999,999,99,NaN,NaN,NaN
15,2080,1,1,16,0,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,6.3,2.2,75,99597,222,1413,346,54,16,52,5972,1127,5905,1290,40,2.6,10,10,9999,99999,9,999999999,100,999,999,99,NaN,NaN,NaN
16,2080,1,1,17,0,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,5.3,2.1,80,99597,35,801,341,7,0,7,758,0,758,155,30,1.7,10,9,9999,99999,9,999999999,100,999,999,99,NaN,NaN,NaN
17,2080,1,1,18,0,*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?*?,4.3,2.1,86,99597,0,0,336,0,0,0,0,0,0,0,10,0.9,10,8,9999,99999,9,999999999,100,999,999,99,NaN,NaN,NaN


In [7]:
# PSB Parking Lot

import math
import pandas as pd
import pvlib
from hoboreader import HoboReader
S2 = HoboReader("DL2_PSB_2020.csv")
df2 = S2.get_dataframe()


!pip install pvlib
# https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.reindl.html?highlight=reindl#pvlib.irradiance.reindl

# conda install -c pvlib pvlib

# Create date series using date_range() function
date_series = pd.date_range('01/01/2019', periods=8760, freq='H')
# print(date_series)

# altitude PSB Station
#altitude = 255

# lat, lon taken from GFR
lon = 42.450620  # north-positive
lat = -76.481201  # west-negative west of england

#pos = pvlib.solarposition.get_solarposition(date_series,lat, lon)
# pos


solar_constant_e = 1367.0
DTR = 0.017453292  # Pi / 180
RTD = 57.2957795  # 180 / Pi


time_zone = 5*15  # west-positive


def month_and_day_to_julian_day(month, day):
    if (month == 1):
        return day
    if (month == 2):
        return day + 31
    if (month == 3):
        return day + 59
    if (month == 4):
        return day + 90
    if (month == 5):
        return day + 120
    if (month == 6):
        return day + 151
    if (month == 7):
        return day + 181
    if (month == 8):
        return day + 212
    if (month == 9):
        return day + 243
    if (month == 10):
        return day + 273
    if (month == 11):
        return day + 304
    if (month == 12):
        return day + 334
    else:
        print("bad month")


def solar_elev_azi_ecc(latitude,
                       longitude,
                       time_zone,
                       jday,
                       time,
                       solar_time=0):
    # /*  angles in degrees, times in hours  */

    # /*  solar elevation and azimuth formulae from sun.c  */
    if (solar_time == 1):
        sol_time = time
    if (solar_time == 0):
        sol_time = time + 0.170 * math.sin(
            (4 * math.pi / 373) * (jday - 80)) - 0.129 * math.sin(
                (2 * math.pi / 355) *
                (jday - 8)) + 12 / 180.0 * (time_zone - longitude)

    solar_declination = RTD * 0.4093 * math.sin(
        (2 * math.pi / 368) * (jday - 81))
    jday_angle = 2 * math.pi * (jday - 1) / 365

    solar_elevation = RTD * math.asin(
        math.sin(latitude * DTR) * math.sin(solar_declination * DTR) -
        math.cos(latitude * DTR) * math.cos(solar_declination * DTR) *
        math.cos(sol_time * (math.pi / 12)))

    solar_azimuth = RTD * (-math.atan2(
        math.cos(solar_declination * DTR) * math.sin(sol_time *
                                                     (math.pi / 12)),
        -math.cos(latitude * DTR) * math.sin(solar_declination * DTR) -
        math.sin(latitude * DTR) * math.cos(solar_declination * DTR) *
        math.cos(sol_time * (math.pi / 12))))

    # /*  eccentricity_correction formula used in genjdaylit.c */

    eccentricity_correction = 1.00011 + 0.034221 * math.cos(
        jday_angle) + 0.00128 * math.sin(jday_angle) + 0.000719 * math.cos(
            2 * jday_angle) + 0.000077 * math.sin(2 * jday_angle)

    return solar_elevation, solar_azimuth, eccentricity_correction


def diffuse_fraction(irrad_glo, solar_elevation, eccentricity_correction):
    dif_frac = 0

    if (solar_elevation > 0):
        irrad_ex = solar_constant_e * eccentricity_correction * math.sin(
            DTR * solar_elevation)
    else:
        irrad_ex = 0

    if (irrad_ex > 0):
        index_glo_ex = irrad_glo / irrad_ex
    else:
        return 0

    if (index_glo_ex < 0):
        print("negative irrad_glo in diffuse_fraction_th\n")
    if (index_glo_ex > 1):
        index_glo_ex = 1

    if (index_glo_ex <= 0.3):
        dif_frac = 1.02 - 0.254 * index_glo_ex + 0.0123 * math.sin(
            DTR * solar_elevation)
    if (dif_frac > 1):
        dif_frac = 1

    if (0.3 < index_glo_ex < 0.78):
        dif_frac = 1.4 - 1.749 * index_glo_ex + 0.177 * math.sin(
            DTR * solar_elevation)
    if (dif_frac > 0.97):
        dif_frac = 0.97
    if (dif_frac < 0.1):
        dif_frac = 0.1

    if (index_glo_ex >= 0.78):
        dif_frac = 0.486 * index_glo_ex - 0.182 * math.sin(
            DTR * solar_elevation)
    if dif_frac < 0.1:
        dif_frac = 0.1

    return dif_frac


def calc_split(month, day, time, irrad_glo):
    jday = month_and_day_to_julian_day(month, day)

    if (irrad_glo < 0 or irrad_glo > solar_constant_e
            ):  # / * check irradiances and exit if necessary * /
        irrad_glo = solar_constant_e

    # no object orientation yet

    solar_elevation, solar_azimuth, eccentricity_correction = solar_elev_azi_ecc(
        lat, lon, time_zone, jday, time)

    #

    irrad_dif = diffuse_fraction(irrad_glo, solar_elevation,
                                 eccentricity_correction) * irrad_glo

    if (solar_elevation > 5.0):

        irrad_beam_nor = (irrad_glo - irrad_dif) * 1.0 / math.sin(
            DTR * solar_elevation)

    else:
        irrad_beam_nor = 0
        irrad_dif = irrad_glo

    if (irrad_beam_nor > solar_constant_e):
        irrad_beam_nor = solar_constant_e
        irrad_dif = irrad_glo - irrad_beam_nor * math.sin(
            DTR * solar_elevation)

    #print(month, day, round(time, 2), round(irrad_beam_nor, 2), round(irrad_dif, 2))

    return irrad_beam_nor, irrad_dif


#print("month, day, time, GHR\n")
calc_split(4, 22, 8.333333333, 107)


def calc_split_df(df):
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['DHR'] = 0
    df['DNI'] = 0

    i = 0
    for index, row in df.iterrows():
        # while i <50:
        GHR = df.iloc[i, df.columns.get_loc('Solar Radiation')].values[0]
        DNI, DHR = calc_split(df.month[i], df.day[i], df.hour[i], GHR)
        print(i,  GHR, DNI, DHR)
        df.iloc[i, df.columns.get_loc('DHR')] = DHR
        df.iloc[i, df.columns.get_loc('DNI')] = DNI
        i += 1

    return df


df2 = calc_split_df(df2)

0 0.6 0.09184481051882247 0.582
1 0.6 0.07777776550384657 0.582
2 0.6 0.06499344614126427 0.582
3 0.6 0.05463690170084893 0.582
4 0.6 0.046711866792790976 0.582
5 0.6 0.040832797546812676 0.582
6 0.6 0.03657719564775516 0.582
7 0.6 0.03360031667403825 0.582
8 3.75 0.19783736308800529 3.6374999999999997
9 11.66666667 0.5945463590501894 11.3166666699
10 50.41666667 2.5443708293804055 48.90416666989999
11 74.35 3.8092249784693903 72.11949999999999
12 72.93333333 3.889096979586593 70.7453333301
13 30.61666667 1.74237963863226 29.6981666699
14 11.46666667 0.7142408801146627 11.1226666699
15 5.6166666670000005 0.39265164761828825 5.448166666990001
16 1.033333333 0.08305643879083988 1.00233333301
17 0.6 0.05664668226184173 0.582
18 0.6 0.06754947953959048 0.582
19 0.6 0.08076384789360613 0.582
20 0.6 0.09477059654184744 0.582
21 0.6 0.1059464732102216 0.582
22 0.6 0.10966801493789788 0.582
23 0.6 0.10402877380339533 0.582
24 0.6 0.09262738154965594 0.582
25 0.6 0.07835914231054474 0.582
26 0.

379 0.6 0.09289463975857207 0.582
380 0.6 0.11306128578554077 0.582
381 0.6 0.13086840691190393 0.582
382 0.6 0.13819008108480638 0.582
383 0.6 0.13058601386890858 0.582
384 0.6 0.11499609393691902 0.582
385 0.6 0.094090009340386 0.582
386 0.6 0.07607736116517033 0.582
387 0.6 0.06221365775615622 0.582
388 0.6 0.05204027966298722 0.582
389 0.6 0.04472850740857582 0.582
390 0.6 0.039551896228128554 0.582
391 2.483333333 0.14890838583497584 2.40883333301
392 32.08333333 1.7991495363112824 31.1208333301
393 95.4 5.140659903609383 92.538
394 170.41666669999998 9.062898307643822 165.30416669899998
395 184.8 10.894924819510809 178.73590441734015
396 182.7166667 11.900655241871156 176.35332458719103
397 291.65 136.13330948936255 223.59340994686883
398 122.71666670000002 8.097776395578874 119.03516669900002
399 62.91666667 4.69636353787578 61.02916666989999
400 17.3 1.5028392188006152 16.781
401 0.6 0.06232593565806875 0.582
402 0.6 0.07622791647904552 0.582
403 0.6 0.09427804275758087 0.582
4

800 11.66666667 0.7382548255663269 11.3166666699
801 45.0 2.7149497212591887 43.65
802 168.9666667 11.375699372163968 163.2135524639571
803 502.71666669999996 768.8393365116008 119.07839062457151
804 559.7833333 799.8577447997656 177.89407878174558
805 547.2833333 808.3816032591611 189.49905367297623
806 329.1666667 464.71955114215854 144.76625060378692
807 62.7 5.4810022763793445 60.819
808 21.23333333 2.2319147171660663 20.5963333301
809 2.2833333330000003 0.3012362562540399 2.21483333301
810 0.6 0.10397898282074945 0.582
811 0.6 0.14257258472967915 0.582
812 0.6 0.19998122464112722 0.582
813 0.6 0 0.6
814 0.6 0 0.6
815 0.6 0 0.6
816 0.6 0 0.6
817 0.6 0.15432955824140532 0.582
818 0.6 0.11078448246435275 0.582
819 0.6 0.08327184622272363 0.582
820 0.6 0.06568844813450253 0.582
821 0.6 0.054197547373457076 0.582
822 0.6 0.046572648989793346 0.582
823 1.45 0.1003601399187293 1.4064999999999999
824 12.51666667 0.7993489852030355 12.141166669899999
825 27.7 1.6856917504262663 26.869
826 

1259 650.6166667 944.8089955182785 268.2819664239125
1260 673.7666667000001 1027.9105359381774 280.5402985540143
1261 641.25 1067.2196815188877 271.1776517259034
1262 502.28333330000004 952.714846314159 216.70797152082085
1263 37.28333333 4.570695345054303 36.164833330099995
1264 21.46666667 3.4735698912565334 20.8226666699
1265 8.733333333 2.081715410425803 8.47133333301
1266 0.6 0 0.6
1267 0.6 0 0.6
1268 0.6 0 0.6
1269 0.6 0 0.6
1270 0.6 0 0.6
1271 0.6 0 0.6
1272 0.6 0 0.6
1273 0.6 0 0.6
1274 0.6 0 0.6
1275 0.6 0.15874309302683234 0.582
1276 0.6 0.10418131380580696 0.582
1277 0.6 0.07745803661510513 0.582
1278 0.816666667 0.08502424005601547 0.79216666699
1279 40.61666667 3.6234382807712686 39.3981666699
1280 161.45 32.452950983693924 149.32905001936183
1281 216.4666667 115.46660505500002 170.59780281874967
1282 448.95 762.1594759542419 139.5676776362543
1283 487.91666669999995 791.4624207886428 172.1678301212347
1284 686.0666667 1060.8181215253803 286.38348957447914
1285 658.1333333

1746 1.016666667 0 1.016666667
1747 0.6 0 0.6
1748 0.6 0 0.6
1749 0.6 0 0.6
1750 0.6 0 0.6
1751 0.6 0 0.6
1752 0.6 0 0.6
1753 0.6 0 0.6
1754 0.6 0 0.6
1755 0.6 0 0.6
1756 0.6 0 0.6
1757 0.6 0.16532970155291557 0.582
1758 3.95 0.7178316229456081 3.8315
1759 34.8 4.895202968796834 33.75599999999999
1760 41.03333333 4.920892449311449 39.8023333301
1761 66.03333333 7.250295677479541 64.0523333301
1762 81.65 8.719727189144372 79.2005
1763 117.91666670000001 42.71427177684244 106.26900865004542
1764 385.65 865.9179914540377 169.94130347500828
1765 318.95 832.1267400130256 142.71564688447629
1766 154.35 725.1999683126195 35.9500433466059
1767 91.26666667 565.1726951142738 30.878687666730382
1768 46.03333333 0 46.03333333
1769 15.83333333 0 15.83333333
1770 0.6 0 0.6
1771 0.6 0 0.6
1772 0.6 0 0.6
1773 0.6 0 0.6
1774 0.6 0 0.6
1775 0.6 0 0.6
1776 0.6 0 0.6
1777 0.6 0 0.6
1778 0.6 0 0.6
1779 0.6 0 0.6
1780 0.6 0 0.6
1781 0.6 0.17573461943833163 0.582
1782 8.95 1.6926967180080905 8.6815
1783 145.

...

6789 105.0 0 105.0
6790 105.0 0 105.0
6791 105.0 0 105.0
6792 105.0 0 105.0
6793 105.0 0 105.0
6794 105.0 0 105.0
6795 105.0 704.2071844970909 41.8157250007471
6796 105.0 324.788487866729 56.214894145037256
6797 105.0 102.0893190814193 83.65606337150815
6798 105.0 20.369609070999523 99.65687452731251
6799 105.0 13.610909003982146 100.83111208202638
6800 105.0 10.361412028377007 101.49765169259844
6801 105.0 8.965596578497388 101.8142437819263
6802 105.0 8.8388671746497 101.84409440572404
6803 105.0 9.93199205020094 101.59279299898617
6804 105.0 12.690726352232229 101.01130220489144
6805 105.0 18.44859144879584 99.96819218707907
6806 105.0 78.75964266841171 87.57620175618375
6807 105.0 251.54502120960262 63.935003732295534
6808 105.0 903.5777702269392 12.139952445447504
6809 105.0 0 105.0
6810 105.0 0 105.0
6811 105.0 0 105.0
6812 105.0 0 105.0
6813 105.0 0 105.0
6814 105.0 0 105.0
6815 105.0 0 105.0
6816 105.0 0 105.0
6817 105.0 0 105.0
6818 105.0 0 105.0
6819 105.0 688.3382512562686 3

7260 105.0 7.502177886231406 101.85
7261 105.0 8.31856782043812 101.85
7262 105.0 11.086660223911455 101.36492183886527
7263 105.0 18.46086840528082 99.9978005183262
7264 105.0 93.06486509136423 85.28877335991609
7265 105.0 292.4884154565233 59.83211650648896
7266 105.0 889.6868171007753 13.586328352389067
7267 105.0 0 105.0
7268 105.0 0 105.0
7269 105.0 0 105.0
7270 105.0 0 105.0
7271 105.0 0 105.0
7272 105.0 0 105.0
7273 105.0 887.6998858395009 13.738110277068008
7274 105.0 297.3895585651026 59.34079088011384
7275 105.0 95.7729217339216 84.85204467356276
7276 105.0 18.711915375083613 99.95807198477006
7277 105.0 11.185143529453164 101.3453527301208
7278 105.0 8.32582065442228 101.85
7279 105.0 7.4869278785711675 101.85
7280 105.0 6.984030675786977 101.85
7281 105.0 6.742823107514939 101.85
7282 105.0 6.7302361763728955 101.85
7283 105.0 6.944590164528563 101.85
7284 105.0 7.41511230618212 101.85
7285 105.0 8.210576874850084 101.85
7286 105.0 10.608774602054496 101.4680705586315
7287 

7683 105.0 15.866117303044202 100.4659377761075
7684 105.0 9.564019976719509 101.7156905504299
7685 105.0 7.887251874314457 101.85
7686 105.0 7.00181185975252 101.85
7687 105.0 6.409271677717787 101.85
7688 105.0 6.042353266838754 101.85
7689 105.0 5.861736782864886 101.85
7690 105.0 5.848813272089434 101.85
7691 105.0 6.002271195359565 101.85
7692 105.0 6.3378570756777926 101.85
7693 105.0 6.891260389802829 101.85
7694 105.0 7.724692864348307 101.85
7695 105.0 8.9381528297345 101.85
7696 105.0 14.6143363927295 100.69184977194108
7697 105.0 47.17402846844502 93.73421554620646
7698 105.0 146.99743739424272 77.31915132687367
7699 105.0 335.3369856424343 55.80055466963839
7700 105.0 630.9407944781985 31.260773040431516
7701 105.0 920.8181224664748 12.173672131081462
7702 105.0 948.3704477536074 10.522257451023798
7703 105.0 682.8080176508188 27.5760000066107
7704 105.0 347.13594779342816 54.67657571310618
7705 105.0 157.51676331174932 75.89265201384053
7706 105.0 53.47197560072367 92.4869

8074 105.0 5.417349983402556 101.85
8075 105.0 5.535480512510206 101.85
8076 105.0 5.803319243520121 101.85
8077 105.0 6.244245078954326 101.85
8078 105.0 6.89755803198985 101.85
8079 105.0 7.822091396637401 101.85
8080 105.0 9.238083155174648 101.80168593901953
8081 105.0 14.975416511229817 100.64146808712448
8082 105.0 42.99652853275664 94.6389900114494
8083 105.0 115.25660555757773 82.01734525292382
8084 105.0 211.7359219659072 69.18025066506658
8085 105.0 295.9273179638695 59.919555395294196
8086 105.0 309.881960264081 58.50384429175181
8087 105.0 241.87136652139577 65.70713740781609
8088 105.0 137.59280714343004 78.75629207794138
8089 105.0 59.17307519390772 91.45177433334014
8090 105.0 17.099898901962163 100.26937197970469
8091 105.0 10.555037269366972 101.51106892768924
8092 105.0 8.139827653859728 101.85
8093 105.0 7.124780189808024 101.85
8094 105.0 6.399996122500586 101.85
8095 105.0 5.901300420988028 101.85
8096 105.0 5.584482716451638 101.85
8097 105.0 5.4219564049571725 10

8469 105.0 183.7101479169846 72.68929306752321
8470 105.0 198.68919241927244 70.8378357105193
8471 105.0 160.65507265852173 75.66586432821002
8472 105.0 96.32806151236524 85.03370301468179
8473 105.0 38.54928972531039 95.59788509613021
8474 105.0 14.992472369712091 100.64860623586641
8475 105.0 9.415363181769527 101.76924915329451
8476 105.0 7.895010241365933 101.85
8477 105.0 6.93858920255836 101.85
8478 105.0 6.245316742254129 101.85
8479 105.0 5.761450428959077 101.85
8480 105.0 5.448184902722634 101.85
8481 105.0 5.280395532329958 101.85
8482 105.0 5.244743564803762 101.85
8483 105.0 5.338411118758829 101.85
8484 105.0 5.5688146988607965 101.85
8485 105.0 5.954345369390589 101.85
8486 105.0 6.525990305105151 101.85
8487 105.0 7.329197401018669 101.85
8488 105.0 8.423861959868661 101.85
8489 105.0 11.60091235994778 101.29999159304168
8490 105.0 18.36117028259511 100.06918128184999
8491 105.0 62.388179983228746 90.88862910579569
8492 105.0 125.4522902886062 80.5682008616263
8493 105.

In [8]:
# PSB_ITHACA STATION

import pandas as pd
from pandas import read_csv
import csv
import unittest
import numpy as np
from pprint import pprint
from setuptools import setup, find_packages
from codecs import open
from os import path
from epw import epw

#S5 = HoboReader("DL5_Game_Farm_Road_2019.csv")
#df5 = S5.get_dataframe()


# Template

template = epw()
template.read('USA_NY_Ithaca_Template.epw')
template.dataframe.columns


# Real file

PSB_2020 = epw()
# PSB_2020.headers['LOCATION'][0]='Ithaca_NY'

PSB_2020.headers = {'LOCATION': ['PSB Parking Lot, Cornell University',
                                 'NY',
                                 'USA',
                                 'TMY3',
                                 '725190',
                                 '42.450620',
                                 '-76.481201',
                                 '-5.0',
                                 '225.0'],
                    'DESIGN CONDITIONS': ['0'],
                    'TYPICAL/EXTREME PERIODS': ['0'],
                    'GROUND TEMPERATURES': ['3',
                                            '0.5',
                                            '',
                                            '',
                                            '',
                                            '2.44',
                                            '4.62',
                                            '9.34',
                                            '15.56',
                                            '21.51',
                                            '25.58',
                                            '26.67',
                                            '24.44',
                                            '19.55',
                                            '13.31',
                                            '7.40',
                                            '3.42',
                                            '2',
                                            '',
                                            '',
                                            '',
                                            '5.55',
                                            '6.08',
                                            '8.78',
                                            '13.07',
                                            '17.76',
                                            '21.58',
                                            '23.50',
                                            '23.98',
                                            '20.18',
                                            '15.85',
                                            '11.17',
                                            '7.40',
                                            '4',
                                            '',
                                            '',
                                            '',
                                            '8.79',
                                            '8.41',
                                            '9.60',
                                            '12.12',
                                            '15.30',
                                            '18.27',
                                            '20.23',
                                            '20.66',
                                            '19.42',
                                            '16.85',
                                            '13.67',
                                            '10.73'],
                    'HOLIDAYS/DAYLIGHT SAVINGS': ['No', '0', '0', '0'],
                    'COMMENTS 1': [''],
                    'COMMENTS 2': [''],
                    'DATA PERIODS': ['1', '1', 'Data', 'Sunday', '1/1', '12/31']}

print(PSB_2020.headers)


#variables = ["Wind Speed","MRT","Temp","RH","Solar Radiation","Wind Speed","Gust Speed","Wind Direction","DewPoint" ]

PSB_2020.dataframe['Year'] = np.array(np.ones(8760)*2019).astype(int)
PSB_2020.dataframe['Month'] = template.dataframe['Month'].astype(int)
PSB_2020.dataframe['Day'] = template.dataframe['Day'].astype(int)
PSB_2020.dataframe['Hour'] = template.dataframe['Hour'].astype(int)
PSB_2020.dataframe['Minute'] = template.dataframe['Minute'].astype(int)
PSB_2020.dataframe['Data Source and Uncertainty Flags'] = np.array(
    np.zeros(8760)).astype(int)


# Actual file starts here
# 6
# .apply(lambda x: ftoc(x), axis=1).values.flatten()
PSB_2020.dataframe['Dry Bulb Temperature'] = df2['Temp'].values.flatten()
# np.array(np.zeros(8760)).astype(int)
PSB_2020.dataframe['Dew Point Temperature'] = df2['DewPt'].values.flatten()


PSB_2020.dataframe['Relative Humidity'] = df2['RH'].values.flatten()
#
PSB_2020.dataframe['Atmospheric Station Pressure'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Extraterrestrial Horizontal Radiation'] = np.array(
    np.zeros(8760)).astype(int)
#
PSB_2020.dataframe['Extraterrestrial Direct Normal Radiation'] = np.array(
    np.zeros(8760)).astype(int)
#
PSB_2020.dataframe['Horizontal Infrared Radiation Intensity'] = np.array(
    np.zeros(8760)).astype(int)
#
PSB_2020.dataframe['Global Horizontal Radiation'] = df2['Solar Radiation'].values.flatten()
PSB_2020.dataframe['Direct Normal Radiation'] = df2['DNI'].values.flatten()
PSB_2020.dataframe['Diffuse Horizontal Radiation'] = df2['DHR'].values.flatten()

# Do we need this?
PSB_2020.dataframe['Global Horizontal Illuminance'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Direct Normal Illuminance'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Diffuse Horizontal Illuminance'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Zenith Luminance'] = np.array(np.zeros(8760)).astype(int)

PSB_2020.dataframe['Wind Direction'] = df2['Wind Direction'].values.flatten(
).astype(int)
PSB_2020.dataframe['Wind Speed'] = df2['Wind Speed'].values.flatten()

#not sure which one to use (skycover)
PSB_2020.dataframe['Total Sky Cover'] = df2['SkyCover'].replace(' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
#     This is from Ithaca airport data (NWS, NOAA) (ADDED TO DL5_2019 FILE 07.22.2020)
PSB_2020.dataframe['Opaque Sky Cover (used if Horizontal IR Intensity missing)'] = df2['SkyCover'].replace(' -', '1.0').astype(float).multiply(10).astype(int).values.flatten()
#

PSB_2020.dataframe['Visibility'] = np.array(np.zeros(8760)).astype(int)
PSB_2020.dataframe['Ceiling Height'] = np.array(np.zeros(8760)).astype(int)
PSB_2020.dataframe['Present Weather Observation'] = np.array(
    np.zeros(8760)).astype(int)
#
PSB_2020.dataframe['Present Weather Codes'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Precipitable Water'] = np.array(np.zeros(8760)).astype(int)
PSB_2020.dataframe['Aerosol Optical Depth'] = np.array(
    np.zeros(8760)).astype(int)
#
PSB_2020.dataframe['Snow Depth'] = np.array(np.zeros(8760)).astype(int)
PSB_2020.dataframe['Days Since Last Snowfall'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Albedo'] = np.array(np.zeros(8760)).astype(int)
#

# We have precipitation but lets not use it for now
PSB_2020.dataframe['Liquid Precipitation Depth'] = np.array(
    np.zeros(8760)).astype(int)
PSB_2020.dataframe['Liquid Precipitation Quantity'] = np.array(
    np.zeros(8760)).astype(int)


print(PSB_2020.dataframe)


PSB_2020.write('PSBLot_2020.epw')

{'LOCATION': ['PSB Parking Lot, Cornell University', 'NY', 'USA', 'TMY3', '725190', '42.450620', '-76.481201', '-5.0', '225.0'], 'DESIGN CONDITIONS': ['0'], 'TYPICAL/EXTREME PERIODS': ['0'], 'GROUND TEMPERATURES': ['3', '0.5', '', '', '', '2.44', '4.62', '9.34', '15.56', '21.51', '25.58', '26.67', '24.44', '19.55', '13.31', '7.40', '3.42', '2', '', '', '', '5.55', '6.08', '8.78', '13.07', '17.76', '21.58', '23.50', '23.98', '20.18', '15.85', '11.17', '7.40', '4', '', '', '', '8.79', '8.41', '9.60', '12.12', '15.30', '18.27', '20.23', '20.66', '19.42', '16.85', '13.67', '10.73'], 'HOLIDAYS/DAYLIGHT SAVINGS': ['No', '0', '0', '0'], 'COMMENTS 1': [''], 'COMMENTS 2': [''], 'DATA PERIODS': ['1', '1', 'Data', 'Sunday', '1/1', '12/31']}
      Year  Month  Day  Hour  Minute  Data Source and Uncertainty Flags  \
0     2019      1    1     1       0                                  0   
1     2019      1    1     2       0                                  0   
2     2019      1    1     3       

[8760 rows x 35 columns]
